In [6]:
import random
import csv

def generate_pid():
    """Generate a realistic PID with meaningful patterns."""
    size = random.randint(100, 999)
    brand = random.randint(100, 999)
    compound = random.randint(100, 999)
    type_code = random.randint(100, 999)
    return f"{size}.{brand}.{compound}.{type_code}"

def get_valid_cycle_time():
    """Generate cycle times in increments of 10 minutes, with most around 75 minutes."""
    # Create a weighted distribution centered around 70-80 minutes
    weights = {
        40: 5,   # Less common
        50: 5,
        60: 10,
        70: 25,  # More common
        80: 25,  # More common
        90: 10,
        100: 5,
        110: 5,
        120: 5,
        130: 3,
        140: 1,
        150: 1
    }

    min_cycle = random.choices(list(weights.keys()), weights=list(weights.values()))[0]

    # Generate maximum cycle time (min_cycle to min_cycle + 50 minutes in steps of 10)
    possible_max = list(range(min_cycle + 10, min(min_cycle + 60, 180), 10))
    if not possible_max:
        possible_max = [min_cycle + 10]
    max_cycle = random.choice(possible_max)

    return min_cycle, max_cycle

def generate_factory_config(filename='factory_config.csv'):
    """Generate factory configuration with realistic constraints."""
    headers = ['oven', 'pid', 'heal', 'soft', 'tread', 'cycle_time_min', 'cycle_time_max']

    configs = []
    for oven in range(1, 33):
        min_cycle, max_cycle = get_valid_cycle_time()

        # All tyres need tread (1)
        heal = random.choice([0, 1])
        soft = random.choice([0, 1])

        config = [
            oven,
            generate_pid(),
            heal,
            soft,
            1,  # tread always required
            min_cycle,
            max_cycle
        ]
        configs.append(config)

    # Write to CSV file
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(configs)

    # Print statistics and analysis
    print(f"Generated configuration for {len(configs)} ovens")

    # Calculate statistics
    heal_required = sum(1 for c in configs if c[2] == 1)
    soft_required = sum(1 for c in configs if c[3] == 1)
    min_cycles = [c[5] for c in configs]
    max_cycles = [c[6] for c in configs]
    avg_min_cycle = sum(min_cycles) / len(min_cycles)
    avg_max_cycle = sum(max_cycles) / len(max_cycles)

    print(f"\nCompound requirements:")
    print(f"Heal required: {(heal_required/len(configs))*100:.1f}%")
    print(f"Soft required: {(soft_required/len(configs))*100:.1f}%")
    print(f"Tread required: 100%")

    print(f"\nCycle times:")
    print(f"Average minimum cycle time: {avg_min_cycle:.1f} minutes")
    print(f"Average maximum cycle time: {avg_max_cycle:.1f} minutes")
    print(f"Overall range: {min(min_cycles)} to {max(max_cycles)} minutes")

    # Distribution analysis
    print("\nCycle time distribution (minimum times):")
    min_distribution = {}
    for time in min_cycles:
        min_distribution[time] = min_distribution.get(time, 0) + 1

    for time, count in sorted(min_distribution.items()):
        print(f"{time} minutes: {count} ovens ({(count/len(configs))*100:.1f}%)")

if __name__ == "__main__":
    generate_factory_config()

Generated configuration for 32 ovens

Compound requirements:
Heal required: 40.6%
Soft required: 50.0%
Tread required: 100%

Cycle times:
Average minimum cycle time: 78.8 minutes
Average maximum cycle time: 105.3 minutes
Overall range: 40 to 160 minutes

Cycle time distribution (minimum times):
40 minutes: 1 ovens (3.1%)
50 minutes: 2 ovens (6.2%)
60 minutes: 3 ovens (9.4%)
70 minutes: 8 ovens (25.0%)
80 minutes: 12 ovens (37.5%)
90 minutes: 2 ovens (6.2%)
110 minutes: 1 ovens (3.1%)
120 minutes: 2 ovens (6.2%)
150 minutes: 1 ovens (3.1%)


In [3]:
!pip install simpy

In [13]:
import simpy
import pandas as pd
import random

class TyreFactory:
    def __init__(self, env, csv_file):
        self.env = env
        # Read configuration from CSV
        self.config = pd.read_csv(csv_file)

        # Resources
        self.heal_mill = simpy.Resource(env, capacity=1)
        self.soft_mill = simpy.Resource(env, capacity=1)
        self.tread_mill = simpy.Resource(env, capacity=1)
        self.ovens = {i: simpy.Resource(env, capacity=1) for i in range(1, 33)}

        # Statistics
        self.completed_tyres = []

    def mill_process(self, tyre_data, mill_type):
        """Simulate the milling process for a specific compound."""
        mill_resource = getattr(self, f"{mill_type}_mill")
        if tyre_data[mill_type] == 1:
            request = mill_resource.request()
            yield request
            process_time = random.uniform(1, 2)  # 1-2 minutes per mill
            yield self.env.timeout(process_time)
            mill_resource.release(request)
            return process_time
        return 0

    def continuous_production(self, oven_num, pid):
        """Continuously produce tyres for a specific oven."""
        while True:  # Keep producing tyres indefinitely
            yield self.env.process(self.handle_tyre(oven_num, pid))
            # No delay between tyres - start next one immediately after current one finishes

    def handle_tyre(self, oven_num, pid):
        """Handle the complete tyre manufacturing process."""
        start_time = self.env.now
        tyre_data = self.config[self.config['oven'] == oven_num].iloc[0]

        # Building Process (Wrapping)
        total_handling_time = 0

        # Process through each mill if required
        mills = ['heal', 'soft', 'tread']
        for mill in mills:
            if tyre_data[mill] == 1:
                mill_resource = getattr(self, f"{mill}_mill")
                request = mill_resource.request()
                yield request
                process_time = random.uniform(1, 2)  # 1-2 minutes per mill
                yield self.env.timeout(process_time)
                mill_resource.release(request)
                total_handling_time += process_time

        # Additional handling time (transport, setup, etc.)
        remaining_handling = random.uniform(
            0,
            max(0, 10 - total_handling_time)  # Ensure total handling time ≤ 10 min
        )
        yield self.env.timeout(remaining_handling)
        total_handling_time += remaining_handling

        # Curing Process
        cycle_time = random.uniform(
            tyre_data['cycle_time_min'],
            tyre_data['cycle_time_max']
        )

        # Request oven
        oven_request = self.ovens[oven_num].request()
        yield oven_request
        yield self.env.timeout(cycle_time)
        self.ovens[oven_num].release(oven_request)

        # Record statistics
        self.completed_tyres.append({
            'pid': pid,
            'oven': oven_num,
            'handling_time': total_handling_time,
            'cycle_time': cycle_time,
            'total_time': total_handling_time + cycle_time,
            'completed_at': self.env.now
        })

    def start_production(self, simulation_time=1440):  # 24 hours = 1440 minutes
        """Start the production simulation."""
        # Start continuous production for each oven
        for _, row in self.config.iterrows():
            self.env.process(self.continuous_production(row['oven'], row['pid']))

def run_simulation(csv_file, simulation_time=1440):
    """Run the factory simulation."""
    env = simpy.Environment()
    factory = TyreFactory(env, csv_file)
    factory.start_production(simulation_time)
    env.run(until=simulation_time)

    # Convert results to DataFrame for analysis
    results_df = pd.DataFrame(factory.completed_tyres)

    # Print summary statistics
    print("\nSimulation Results:")
    print(f"Total tyres produced: {len(results_df)}")
    if not results_df.empty:
        print(f"Average production rate: {len(results_df)/24:.1f} tyres per hour")
        print(f"Average handling time: {results_df['handling_time'].mean():.2f} minutes")
        print(f"Average cycle time: {results_df['cycle_time'].mean():.2f} minutes")
        print(f"Average total time: {results_df['total_time'].mean():.2f} minutes")

        # Group by oven to see production statistics
        oven_stats = results_df.groupby('oven').agg({
            'pid': 'count',
            'handling_time': 'mean',
            'cycle_time': 'mean',
            'total_time': 'mean'
        }).round(2)
        oven_stats.columns = ['Tyres Produced', 'Avg Handling Time', 'Avg Cycle Time', 'Avg Total Time']

        print("\nProduction by Oven:")
        print(oven_stats)

        # Group by PID to see production statistics per tyre type
        pid_stats = results_df.groupby('pid').agg({
            'handling_time': ['mean', 'min', 'max'],
            'cycle_time': ['mean', 'min', 'max'],
            'total_time': ['mean', 'min', 'max']
        }).round(2)

        print("\nStatistics by PID:")
        print(pid_stats)

    return results_df

# Example usage:
if __name__ == "__main__":
    csv_file = "factory_config.csv"
    results = run_simulation(csv_file)


Simulation Results:
Total tyres produced: 463
Average production rate: 19.3 tyres per hour
Average handling time: 6.41 minutes
Average cycle time: 87.27 minutes
Average total time: 93.67 minutes

Production by Oven:
      Tyres Produced  Avg Handling Time  Avg Cycle Time  Avg Total Time
oven                                                                   
1                 17               6.12           74.70           80.83
2                 12               7.18          100.81          108.00
3                 10               5.37          136.43          141.80
4                 20               6.32           63.86           70.17
5                 11               5.69          115.63          121.32
6                 18               7.11           70.51           77.61
7                 13               6.05          100.23          106.28
8                 23               5.48           54.69           60.17
9                 19               6.48           65.19        

In [15]:
import simpy
import pandas as pd
import random

class TyreFactory:
    def __init__(self, env, csv_file):
        self.env = env
        self.config = pd.read_csv(csv_file)

        # Resources
        self.heal_mill = simpy.Resource(env, capacity=1)
        self.soft_mill = simpy.Resource(env, capacity=1)
        self.tread_mill = simpy.Resource(env, capacity=1)
        self.ovens = {i: simpy.Resource(env, capacity=1) for i in range(1, 33)}

        # Statistics
        self.completed_tyres = []
        self.wait_times = {
            'heal_mill': [],
            'soft_mill': [],
            'tread_mill': [],
            'ovens': []
        }

    def mill_process(self, tyre_data, mill_type):
        """Simulate the milling process with wait time tracking."""
        mill_resource = getattr(self, f"{mill_type}_mill")
        if tyre_data[mill_type] == 1:
            arrive_time = self.env.now
            request = mill_resource.request()
            yield request

            # Calculate and record wait time
            wait_time = self.env.now - arrive_time
            self.wait_times[f'{mill_type}_mill'].append(wait_time)

            process_time = random.uniform(1, 2)
            yield self.env.timeout(process_time)
            mill_resource.release(request)
            return process_time, wait_time
        return 0, 0

    def continuous_production(self, oven_num, pid):
        """Continuously produce tyres for a specific oven."""
        while True:
            yield self.env.process(self.handle_tyre(oven_num, pid))

    def handle_tyre(self, oven_num, pid):
        """Handle the complete tyre manufacturing process with wait time tracking."""
        start_time = self.env.now
        tyre_data = self.config[self.config['oven'] == oven_num].iloc[0]

        # Track wait times for each stage
        total_handling_time = 0
        wait_times = {'heal': 0, 'soft': 0, 'tread': 0, 'oven': 0}
        process_times = {'heal': 0, 'soft': 0, 'tread': 0}

        # Process through each mill if required
        mills = ['heal', 'soft', 'tread']
        for mill in mills:
            if tyre_data[mill] == 1:
                process_time, wait_time = yield self.env.process(self.mill_process(tyre_data, mill))
                total_handling_time += process_time
                wait_times[mill] = wait_time
                process_times[mill] = process_time

        # Additional handling time (transport, setup)
        remaining_handling = random.uniform(0, max(0, 10 - total_handling_time))
        yield self.env.timeout(remaining_handling)
        total_handling_time += remaining_handling

        # Curing Process
        cycle_time = random.uniform(
            tyre_data['cycle_time_min'],
            tyre_data['cycle_time_max']
        )

        # Oven process with wait time tracking
        oven_arrive_time = self.env.now
        oven_request = self.ovens[oven_num].request()
        yield oven_request
        oven_wait_time = self.env.now - oven_arrive_time
        self.wait_times['ovens'].append(oven_wait_time)
        wait_times['oven'] = oven_wait_time

        yield self.env.timeout(cycle_time)
        self.ovens[oven_num].release(oven_request)

        # Record detailed statistics
        self.completed_tyres.append({
            'pid': pid,
            'oven': oven_num,
            'handling_time': total_handling_time,
            'cycle_time': cycle_time,
            'total_time': total_handling_time + cycle_time,
            'completed_at': self.env.now,
            'heal_wait': wait_times['heal'],
            'soft_wait': wait_times['soft'],
            'tread_wait': wait_times['tread'],
            'oven_wait': wait_times['oven'],
            'total_wait': sum(wait_times.values()),
            'heal_process': process_times['heal'],
            'soft_process': process_times['soft'],
            'tread_process': process_times['tread']
        })

    def start_production(self, simulation_time=1440):
        """Start the production simulation."""
        for _, row in self.config.iterrows():
            self.env.process(self.continuous_production(row['oven'], row['pid']))

def run_simulation(csv_file, simulation_time=1440):
    """Run the factory simulation with detailed wait time analysis."""
    env = simpy.Environment()
    factory = TyreFactory(env, csv_file)
    factory.start_production(simulation_time)
    env.run(until=simulation_time)

    results_df = pd.DataFrame(factory.completed_tyres)

    print("\n=== Simulation Results ===")
    print(f"Total tyres produced: {len(results_df)}")
    if not results_df.empty:
        print(f"Production rate: {len(results_df)/24:.1f} tyres per hour")

        print("\n=== Wait Time Analysis ===")
        print("\nAverage Wait Times (minutes):")
        print(f"Heal Mill: {results_df['heal_wait'].mean():.2f}")
        print(f"Soft Mill: {results_df['soft_wait'].mean():.2f}")
        print(f"Tread Mill: {results_df['tread_wait'].mean():.2f}")
        print(f"Ovens: {results_df['oven_wait'].mean():.2f}")
        print(f"Total Average Wait: {results_df['total_wait'].mean():.2f}")

        print("\nMaximum Wait Times (minutes):")
        print(f"Heal Mill: {results_df['heal_wait'].max():.2f}")
        print(f"Soft Mill: {results_df['soft_wait'].max():.2f}")
        print(f"Tread Mill: {results_df['tread_wait'].max():.2f}")
        print(f"Ovens: {results_df['oven_wait'].max():.2f}")

        print("\n=== Resource Utilization ===")
        total_time = simulation_time
        # Calculate utilization based on process times
        for resource in ['heal', 'soft', 'tread']:
            total_process_time = results_df[f'{resource}_process'].sum()
            utilization = (total_process_time / total_time) * 100
            print(f"{resource.capitalize()} Mill Utilization: {utilization:.1f}%")

        print("\n=== Bottleneck Analysis ===")
        wait_columns = ['heal_wait', 'soft_wait', 'tread_wait', 'oven_wait']
        bottleneck = wait_columns[results_df[wait_columns].mean().argmax()]
        print(f"Primary bottleneck: {bottleneck.replace('_wait', '')} "
              f"(Average wait: {results_df[bottleneck].mean():.2f} minutes)")

        # Detailed oven statistics
        print("\n=== Oven Performance ===")
        oven_stats = results_df.groupby('oven').agg({
            'pid': 'count',
            'total_wait': 'mean',
            'cycle_time': 'mean',
            'total_time': 'mean'
        }).round(2)
        oven_stats.columns = ['Tyres Produced', 'Avg Wait Time', 'Avg Cycle Time', 'Avg Total Time']
        print(oven_stats)

    return results_df

# Example usage:
if __name__ == "__main__":
    csv_file = "factory_config.csv"
    results = run_simulation(csv_file)


=== Simulation Results ===
Total tyres produced: 465
Production rate: 19.4 tyres per hour

=== Wait Time Analysis ===

Average Wait Times (minutes):
Heal Mill: 0.33
Soft Mill: 0.28
Tread Mill: 1.71
Ovens: 0.00
Total Average Wait: 2.32

Maximum Wait Times (minutes):
Heal Mill: 17.53
Soft Mill: 11.64
Tread Mill: 24.17
Ovens: 0.00

=== Resource Utilization ===
Heal Mill Utilization: 20.6%
Soft Mill Utilization: 25.8%
Tread Mill Utilization: 49.1%

=== Bottleneck Analysis ===
Primary bottleneck: tread (Average wait: 1.71 minutes)

=== Oven Performance ===
      Tyres Produced  Avg Wait Time  Avg Cycle Time  Avg Total Time
oven                                                               
1                 17           1.70           75.82           81.89
2                 12           2.00          102.98          108.93
3                 10           0.57          135.64          141.53
4                 19           2.54           64.57           70.91
5                 11           2.

In [16]:
import simpy
import pandas as pd
import random
from datetime import datetime, timedelta

class TyreFactory:
    def __init__(self, env, csv_file):
        self.env = env
        self.config = pd.read_csv(csv_file)

        # Resources
        self.heal_mill = simpy.Resource(env, capacity=1)
        self.soft_mill = simpy.Resource(env, capacity=1)
        self.tread_mill = simpy.Resource(env, capacity=1)
        self.ovens = {i: simpy.Resource(env, capacity=1) for i in range(1, 33)}

        # Statistics
        self.completed_tyres = []
        self.oven_outputs = []  # Track oven output times
        self.wait_times = {
            'heal_mill': [],
            'soft_mill': [],
            'tread_mill': [],
            'ovens': []
        }

    def mill_process(self, tyre_data, mill_type):
        """Simulate the milling process with wait time tracking."""
        mill_resource = getattr(self, f"{mill_type}_mill")
        if tyre_data[mill_type] == 1:
            arrive_time = self.env.now
            request = mill_resource.request()
            yield request
            wait_time = self.env.now - arrive_time
            self.wait_times[f'{mill_type}_mill'].append(wait_time)
            process_time = random.uniform(1, 2)
            yield self.env.timeout(process_time)
            mill_resource.release(request)
            return process_time, wait_time
        return 0, 0

    def continuous_production(self, oven_num, pid):
        """Continuously produce tyres for a specific oven."""
        while True:
            yield self.env.process(self.handle_tyre(oven_num, pid))

    def handle_tyre(self, oven_num, pid):
        """Handle the complete tyre manufacturing process with detailed timing."""
        start_time = self.env.now
        tyre_data = self.config[self.config['oven'] == oven_num].iloc[0]

        # Track timing for each stage
        total_handling_time = 0
        wait_times = {'heal': 0, 'soft': 0, 'tread': 0, 'oven': 0}
        process_times = {'heal': 0, 'soft': 0, 'tread': 0}

        # Process through each mill if required
        mills = ['heal', 'soft', 'tread']
        for mill in mills:
            if tyre_data[mill] == 1:
                process_time, wait_time = yield self.env.process(self.mill_process(tyre_data, mill))
                total_handling_time += process_time
                wait_times[mill] = wait_time
                process_times[mill] = process_time

        # Additional handling time
        remaining_handling = random.uniform(0, max(0, 10 - total_handling_time))
        yield self.env.timeout(remaining_handling)
        total_handling_time += remaining_handling

        # Curing Process
        cycle_time = random.uniform(
            tyre_data['cycle_time_min'],
            tyre_data['cycle_time_max']
        )

        oven_arrive_time = self.env.now
        oven_request = self.ovens[oven_num].request()
        yield oven_request
        oven_wait_time = self.env.now - oven_arrive_time

        # Record oven entry time
        oven_entry_time = self.env.now

        yield self.env.timeout(cycle_time)
        oven_exit_time = self.env.now

        # Record oven output details
        self.oven_outputs.append({
            'oven': oven_num,
            'pid': pid,
            'entry_time': oven_entry_time,
            'exit_time': oven_exit_time,
            'cycle_time': cycle_time
        })

        self.ovens[oven_num].release(oven_request)

        # Record complete tyre details
        self.completed_tyres.append({
            'pid': pid,
            'oven': oven_num,
            'handling_time': total_handling_time,
            'cycle_time': cycle_time,
            'total_time': total_handling_time + cycle_time,
            'completed_at': self.env.now,
            'heal_wait': wait_times['heal'],
            'soft_wait': wait_times['soft'],
            'tread_wait': wait_times['tread'],
            'oven_wait': oven_wait_time,
            'total_wait': sum(wait_times.values()) + oven_wait_time,
            'heal_process': process_times['heal'],
            'soft_process': process_times['soft'],
            'tread_process': process_times['tread']
        })

    def start_production(self, simulation_time=1440):
        """Start the production simulation."""
        for _, row in self.config.iterrows():
            self.env.process(self.continuous_production(row['oven'], row['pid']))

def format_time(minutes):
    """Convert simulation minutes to HH:MM format."""
    hours = int(minutes // 60)
    mins = int(minutes % 60)
    return f"{hours:02d}:{mins:02d}"

def run_simulation(csv_file, simulation_time=1440):
    """Run the factory simulation with detailed timing analysis."""
    env = simpy.Environment()
    factory = TyreFactory(env, csv_file)
    factory.start_production(simulation_time)
    env.run(until=simulation_time)

    results_df = pd.DataFrame(factory.completed_tyres)
    oven_outputs_df = pd.DataFrame(factory.oven_outputs)

    print("\n=== Simulation Results ===")
    print(f"Total tyres produced: {len(results_df)}")
    print(f"Production rate: {len(results_df)/24:.1f} tyres per hour")

    print("\n=== Oven Output Times ===")
    # Sort oven outputs by oven number and exit time
    oven_outputs_df = oven_outputs_df.sort_values(['oven', 'exit_time'])

    # Group by oven and show output details
    for oven in range(1, 33):
        oven_data = oven_outputs_df[oven_outputs_df['oven'] == oven]
        if not oven_data.empty:
            print(f"\nOven {oven} outputs:")
            print(f"Total tyres: {len(oven_data)}")
            print("First 5 outputs:")
            for _, row in oven_data.head().iterrows():
                print(f"  PID: {row['pid']}, "
                      f"Entry: {format_time(row['entry_time'])}, "
                      f"Exit: {format_time(row['exit_time'])}, "
                      f"Cycle time: {row['cycle_time']:.1f} min")

    print("\n=== Resource Utilization ===")
    total_time = simulation_time
    for resource in ['heal', 'soft', 'tread']:
        total_process_time = results_df[f'{resource}_process'].sum()
        utilization = (total_process_time / total_time) * 100
        print(f"{resource.capitalize()} Mill Utilization: {utilization:.1f}%")

    print("\n=== Oven Performance Summary ===")
    oven_stats = results_df.groupby('oven').agg({
        'pid': 'count',
        'cycle_time': ['mean', 'min', 'max'],
        'total_wait': 'mean'
    }).round(2)
    print(oven_stats)

    return results_df, oven_outputs_df

# Example usage:
if __name__ == "__main__":
    csv_file = "factory_config.csv"
    results, oven_outputs = run_simulation(csv_file)


=== Simulation Results ===
Total tyres produced: 463
Production rate: 19.3 tyres per hour

=== Oven Output Times ===

Oven 1 outputs:
Total tyres: 17
First 5 outputs:
  PID: 696.745.426.845, Entry: 00:18, Exit: 01:37, Cycle time: 78.7 min
  PID: 696.745.426.845, Entry: 01:45, Exit: 03:01, Cycle time: 75.9 min
  PID: 696.745.426.845, Entry: 03:09, Exit: 04:25, Cycle time: 75.5 min
  PID: 696.745.426.845, Entry: 04:33, Exit: 05:49, Cycle time: 76.0 min
  PID: 696.745.426.845, Entry: 05:57, Exit: 07:12, Cycle time: 75.2 min

Oven 2 outputs:
Total tyres: 12
First 5 outputs:
  PID: 861.900.132.135, Entry: 00:25, Exit: 01:56, Cycle time: 91.1 min
  PID: 861.900.132.135, Entry: 02:04, Exit: 03:37, Cycle time: 92.9 min
  PID: 861.900.132.135, Entry: 03:44, Exit: 05:28, Cycle time: 104.5 min
  PID: 861.900.132.135, Entry: 05:35, Exit: 07:09, Cycle time: 93.6 min
  PID: 861.900.132.135, Entry: 07:18, Exit: 09:14, Cycle time: 116.4 min

Oven 3 outputs:
Total tyres: 10
First 5 outputs:
  PID: 283